In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# About

**Mobile Topup (Topup service)** is one of MoMo's most important services with a large proportion of users and revenue. Topup service on the MoMo app is displayed under two icons: 'Nạp tiền điện thoại' and 'Mua mã thẻ di động'. The data give the historical daily transactions data during Jan-Dec 2020 (table 'Transactions'), the users' demographic information (table 'User_info'), and the percentage of commission paid by the Telco's merchants to MoMo (table 'Commission'). 

# Loading Data 

In [2]:
data_path = '/kaggle/input/mobile-top/MoMo Talent 2024_DA_Case Study Round_Questions.xlsx'
all_sheets = pd.read_excel(data_path, sheet_name = None)

In [3]:
# In ra tên các sheet
print(all_sheets.keys())

dict_keys(['Questions', 'Data Transactions', 'Data Commission', 'Data User_Info'])


In [4]:
transactions = all_sheets['Data Transactions']
commission = all_sheets['Data Commission']
user_Info = all_sheets[ 'Data User_Info']

# Data Processing

## Data Transactions

| # | Attribute | Describe
| --- | --- | --- : 
| 1 | User_id | each user in MoMo will be given a unique id
| 2 | Order_id | each transaction will be given a unique id
| 3 | Date | date when the transaction took place
| 4 | Amount | amount of money that user spent on that transaction (VND)
| 5 | Merchant_id | each merchant will be given a unique id
| 6 | Purchase_status | if the user uses the account to buy a card for another user, it will be labeled 'Mua hộ'

In [5]:
transactions.head()

,user_id,order_id,Date,Amount,Merchant_id,Purchase_status
0,21269588,4169517626,2020-01-01,"10,000",13,NaN
1,28097592,4170276686,2020-01-01,"20,000",13,NaN
2,47435144,4166729310,2020-01-01,"10,000",12,NaN
3,29080935,4174460303,2020-01-01,"10,000",13,NaN
4,14591075,4168216749,2020-01-01,"10,000",12,NaN


In [6]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          13495 non-null  int64 
 1   order_id         13495 non-null  int64 
 2   Date             13495 non-null  object
 3   Amount           13495 non-null  object
 4   Merchant_id      13495 non-null  int64 
 5   Purchase_status  2235 non-null   object
dtypes: int64(3), object(3)
memory usage: 632.7+ KB


**From the above results, I can conclude and note the following:**
- The "Purchase_status" column currently lacks values. This is evident because, following its definition, it is only labeled 'Mua hộ' when a user buys a card for another person. To avoid confusion, I propose labeling it "Chính chủ" when the user buys a card for themselves.
- Some columns have incorrect data types:
    - "user_id", "order_id", and "Merchant_id" should be converted to **object** instead of **int64**.
    - "Date" should be converted to **date** type instead of **object**.
    - "Amount" should be converted to **int64** instead of **object**.

In [7]:
# Add additional labeling for the "Purchase Status" column by converting NaN values to "Chính chủ"
transactions['Purchase_status'] = transactions['Purchase_status'].fillna('Chính chủ')

In [8]:
# convert int64 to object
transactions['user_id'] = transactions['user_id'].astype(object)
transactions['order_id'] = transactions['order_id'].astype(object)
transactions[ 'Merchant_id'] = transactions['Merchant_id'].astype(object)

In [9]:
# convert object to float
# The values in the Amount column contain commas so their data type cannot be converted directly to the float. 
# We need to change the commas to space and then convert this data type to float.
transactions['Amount'] = transactions['Amount'].str.replace(',', '').astype(float)

In [10]:
# convert object to date
# When I did the object to date conversion an error occurred. 
# Values are out of sync with each other such as "2020-01-01", "27/9/2020"
from dateutil.parser import parse, ParserError
def is_date_format_consistent(date_str):
    try:
        pd.to_datetime(date_str, format="%Y-%m-%d")
        return True
    except:
        return False
    
# Check the date consistency of all samples
transactions['Date_Check'] = transactions['Date'].apply(is_date_format_consistent)
print(transactions['Date_Check'].value_counts())

Date_Check
True     13269
False      226
Name: count, dtype: int64


In [11]:
valid_dates  = transactions[transactions['Date_Check'] == True].copy()
valid_dates['Date'] = pd.to_datetime(valid_dates['Date'] , format="%Y-%m-%d") 

invalid_dates = transactions[~transactions['Date_Check']].copy()
invalid_dates['Date'] = pd.to_datetime(invalid_dates['Date'], format="%d/%m/%Y")
invalid_dates['Date']= invalid_dates['Date'].apply(lambda x: x.strftime("%Y-%m-%d"))

transactions = pd.concat([valid_dates, invalid_dates]).drop(columns="Date_Check")
transactions['Date'] = pd.to_datetime(transactions['Date'], errors='coerce')

In [12]:
# Check missing values
missing_values = transactions.isnull().sum().sum()
print('A number of missing values in the transaction data: ', missing_values)
print('The percentage of missing values in the transaction data: ', (missing_values/len(transactions))*100, '%')

A number of missing values in the transaction data:  0
The percentage of missing values in the transaction data:  0.0 %


In [13]:
# check duplicate samples
duplicated_samples = transactions.duplicated().sum()
print('A number of duplicated samples: ',duplicated_samples)

A number of duplicated samples:  0


In [14]:
# Check unique values of this column
transactions['Amount'].unique()

array([  10000.,   20000.,   50000.,   30000.,  100000.,  200000.,
        500000.,   40000.,  294234.,  300000.,  400000., 1000000.,
       2000000.,   60000., 2500000.,  250000.])

**From the above results, I can conclude and note the following:**
- Most denominations of money in transactions are round numbers such as 10k, 20k, 100k, 250k, 300k, and so on. However, there is one denomination that is not synchronized with the remaining denominations, which is 294,234. I propose aligning this denomination with the others by rounding it up to 300,000.

In [15]:
transactions.loc[transactions['Amount'] == 294234, 'Amount'] = 300000

## Data Commission

| # | Attribute | Describe
| --- | --- | --- : 
| 1 | Merchant_name | name of Telco
| 2 | Merchant_id | each merchant will be given a unique id
| 3 | Rate_pct | commission percentage (%) that MoMo earns from the merchant for each transaction amount

In [16]:
commission.head()

,Merchant_name,Merchant_id,Rate_pct
0,Viettel,12,2
1,Mobifone,13,3
2,Vinaphone,14,4
3,Vietnamobile,15,4
4,Gmobile,16,4


In [17]:
commission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Merchant_name  5 non-null      object
 1   Merchant_id    5 non-null      int64 
 2   Rate_pct       5 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 248.0+ bytes


**From the above results, I can conclude and note the following:**
- The "Merchant_id" column is incorrectly data types, it should be converted to **object** instead of **int64**.
  

In [18]:
# covert int64 to object
commission['Merchant_id'] = commission['Merchant_id'].astype(object)

## Data User_Info

In [19]:
user_Info.head()

,User_id,First_tran_date,Location,Age,Gender
0,41654498,2018-05-02,HCMC,33_to_37,MALE
1,51276281,2019-12-08,Other Cities,unknown,MALE
2,49152375,2019-12-20,Other Cities,23_to_27,MALE
3,5971050,2019-08-01,HCMC,28_to_32,MALE
4,48134464,2019-08-23,Other Cities,18_to_22,FEMALE


In [20]:
user_Info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13428 entries, 0 to 13427
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User_id          13428 non-null  int64 
 1   First_tran_date  13428 non-null  object
 2   Location         13428 non-null  object
 3   Age              13428 non-null  object
 4   Gender           13428 non-null  object
dtypes: int64(1), object(4)
memory usage: 524.7+ KB


**From the above results, we can conclude and note the following:**
- Some columns have incorrect data types:
    - "User_id" should be converted to **object** instead of **int64**.
    - "First_tran_date" should be converted to **date** type instead of **object**.

In [21]:
# convert int64 to object
user_Info['User_id'] = user_Info['User_id'].astype(object)

In [22]:
# Check the date consistency of all samples
user_Info['Date_Check'] = user_Info['First_tran_date'].apply(is_date_format_consistent)
user_Info['Date_Check'].value_counts()

Date_Check
True     13354
False       74
Name: count, dtype: int64

This case is not the same as the case in the transaction data, where some data has dates like **3020-05-28**, **9919-07-02**, **9920-03-05**, etc. Transactions in these years are unreasonable and may be an error during data entry. To address this issue, I propose converting the "30" and "99" to "20" in the year portion of these dates.

In [23]:
# Convert the valid Date column to the datetime data type
valid_date = user_Info[user_Info['Date_Check'] == True].copy()
valid_date['First_tran_date'] = pd.to_datetime(valid_date['First_tran_date'] , format="%Y-%m-%d") 

invalid_date = user_Info[~user_Info['Date_Check']].copy()
# Replace "30", "99" to "20"
invalid_date['First_tran_date'] = invalid_date['First_tran_date'].str.replace(r"^30", "20", regex=True)
invalid_date['First_tran_date'] = invalid_date['First_tran_date'].str.replace(r"^99", "20", regex=True)

# Convert the Date column to the datetime data type
invalid_date['First_tran_date'] = pd.to_datetime(invalid_date['First_tran_date'], format="%Y-%m-%d")

# Concat two data (valid_date,invalid_date) to create a new user_Infor fixed all errors
user_Info = pd.concat([valid_date,invalid_date]).drop(columns = 'Date_Check')

In [24]:
# The information data after fixing
user_Info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13428 entries, 0 to 13384
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   User_id          13428 non-null  object        
 1   First_tran_date  13428 non-null  datetime64[ns]
 2   Location         13428 non-null  object        
 3   Age              13428 non-null  object        
 4   Gender           13428 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 629.4+ KB


### Check the "Location", "Age" and "Gender" columns

#### **Age column**

In [25]:
# Check unique values of this column
print('Unique values: ',user_Info['Age'].unique())

Unique values:  ['33_to_37' 'unknown' '23_to_27' '28_to_32' '18_to_22' '>37']


In [26]:
# Replace "unknown" value to NaN value
user_Info['Age'] = user_Info['Age'].replace('unknown', np.nan)

#### **Location column**

In [27]:
# Check unique values of this column
user_Info['Location'].unique()

array(['HCMC', 'Other Cities', 'Other', 'HN', 'Unknown',
       'Ho Chi Minh City'], dtype=object)

**From the above results, we can conclude and note the following:**
- "HCMC" and "Ho Chi Minh City" refer to locations in Ho Chi Minh City, so I will group them as "HCMC"
- "Other" and "Other Cities" refer to cities, so I will group them as "Other Cities"
- "Unknown" value means unknown location of user, so I will replace them to NaN value

In [28]:
replace_dict = {
    'Other Cities': 'Other Cities',
    'Other': 'Other Cities',
    'HCMC': 'HCMC',
    'Ho Chi Minh City': 'HCMC',
    'HN': 'HN',
    'Unknown':np.nan
}
user_Info['Location'] = user_Info['Location'].replace(replace_dict)

#### **Gender column**

In [29]:
# Check unique values of this column
user_Info['Gender'].unique()

array(['MALE', 'FEMALE', 'Nữ', 'M', 'female', 'Nam', 'f', 'male'],
      dtype=object)

**From the above results, we can conclude and note the following:**
- "female", "F" , "Nữ", and "FEMALE" refer to female sex, so I will group them as "F"
- "male", "M", "Nam" and "MALE" refer to female sex, so I will group them as "M"


In [30]:
replace_dict = {
    'female':'F',
    'FEMALE':'F',
    'f':'F',
    'Nữ':'F',
    'male':'M',
    'MALE':'M',
    'M':'M',
    'Nam':'M'
}
user_Info['Gender'] = user_Info['Gender'].replace(replace_dict)

In [31]:
# Check missing values
missing_values = user_Info.isnull().sum().sum()
print('A number of missing values in the transaction data: ', missing_values)
print('The percentage of missing values in the transaction data: ', (missing_values/len(user_Info))*100, '%')
print('=> The percentage  of missing values is quite large. I need to consider how to handle them')

A number of missing values in the transaction data:  3282
The percentage of missing values in the transaction data:  24.441465594280608 %
=> The percentage  of missing values is quite large. I need to consider how to handle them


In [32]:
# check duplicate samples
duplicated_samples = transactions.duplicated().sum()
print('A number of duplicated samples: ',duplicated_samples)

A number of duplicated samples:  0


In [33]:
def Rate_pct(mechant_id):
    return commission[commission['Merchant_id'] == mechant_id ]['Rate_pct'].iloc[0]

rate_pct = transactions['Merchant_id'].apply(Rate_pct)

In [34]:
# Add a column 'Revenue' in the 'Transactions' table 
transactions['revenue'] = transactions['Amount']*(rate_pct/100)
transactions.head()

,user_id,order_id,Date,Amount,Merchant_id,Purchase_status,revenue
0,21269588,4169517626,2020-01-01,10000.0,13,Chính chủ,300.0
1,28097592,4170276686,2020-01-01,20000.0,13,Chính chủ,600.0
2,47435144,4166729310,2020-01-01,10000.0,12,Chính chủ,200.0
3,29080935,4174460303,2020-01-01,10000.0,13,Chính chủ,300.0
4,14591075,4168216749,2020-01-01,10000.0,12,Chính chủ,200.0


In [35]:
# Calculate MoMo's total revenue in January 2020. 
january20_revenue = transactions[(transactions['Date'].dt.year == 2020) & (transactions['Date'].dt.month == 1)]['revenue'].sum()
print(" MoMo's total revenue in January 2020: ", january20_revenue)

 MoMo's total revenue in January 2020:  1410000.0


In [36]:
# Convert 'Date' to month period and calculate monthly revenue
transactions['Month'] = transactions['Date'].dt.to_period('M')
monthly_revenue = transactions.groupby('Month')['revenue'].sum()

#  Find most profitable month
most_profitable_month = monthly_revenue.idxmax()
max_revenue = monthly_revenue.max()

print("Most profitable month:", most_profitable_month)
print("Revenue for the most profitable month:", max_revenue)

Most profitable month: 2020-09
Revenue for the most profitable month: 1702200.0


In [37]:
# The revenue by day of the week 
transactions['DayOfWeek'] = transactions['Date'].dt.day_name()
weekly_revenue = transactions.groupby('DayOfWeek')['revenue'].mean().reset_index()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekly_revenue['DayOfWeek'] = pd.Categorical(weekly_revenue['DayOfWeek'], categories=day_order, ordered=True)
weekly_revenue = weekly_revenue.sort_values('DayOfWeek')
print(weekly_revenue)

   DayOfWeek      revenue
1     Monday  1304.437564
5    Tuesday  1401.358412
6  Wednesday  1482.535075
4   Thursday  1411.190598
0     Friday  1408.213041
2   Saturday  1376.099946
3     Sunday  1308.140192


In [38]:
max_revenue_day = weekly_revenue.loc[weekly_revenue['revenue'].idxmax(), 'DayOfWeek']
min_revenue_day = weekly_revenue.loc[weekly_revenue['revenue'].idxmin(), 'DayOfWeek']

print('The day of the week make the most money: ', max_revenue_day)
print('The day of the week make the least money: ', min_revenue_day)

The day of the week make the most money:  Wednesday
The day of the week make the least money:  Monday


### Combined with the 'User_Info' table, add columns: Age, Gender, Location, Type_user (New/Current) in 'Transactions' table. 
Note: New = the transaction was in the month of the first time the user used Topup; Current = the user had used Topup before that month

In [39]:
merged_df = pd.merge(transactions, user_Info,  left_on='user_id', right_on='User_id', how='left')
merged_df = merged_df.drop(columns = 'User_id')
merged_df.head()

,user_id,order_id,Date,Amount,Merchant_id,Purchase_status,revenue,Month,DayOfWeek,First_tran_date,Location,Age,Gender
0,21269588,4169517626,2020-01-01,10000.0,13,Chính chủ,300.0,2020-01,Wednesday,2018-12-11,HN,>37,F
1,28097592,4170276686,2020-01-01,20000.0,13,Chính chủ,600.0,2020-01,Wednesday,2019-12-30,HN,>37,F
2,47435144,4166729310,2020-01-01,10000.0,12,Chính chủ,200.0,2020-01,Wednesday,2019-11-11,HN,18_to_22,F
3,29080935,4174460303,2020-01-01,10000.0,13,Chính chủ,300.0,2020-01,Wednesday,2019-10-24,HN,18_to_22,F
4,14591075,4168216749,2020-01-01,10000.0,12,Chính chủ,200.0,2020-01,Wednesday,2019-12-28,Other Cities,18_to_22,F


In [40]:
def compare_dates(row):
    if row['Date'] > row['First_tran_date']:
        return 'current'
    else:
        return 'new'

# Apply the function to create the new column 'type_user'
merged_df['type_user'] = merged_df.apply(lambda row: compare_dates(row), axis=1)

In [41]:
merged_df.head()

,user_id,order_id,Date,Amount,Merchant_id,Purchase_status,revenue,Month,DayOfWeek,First_tran_date,Location,Age,Gender,type_user
0,21269588,4169517626,2020-01-01,10000.0,13,Chính chủ,300.0,2020-01,Wednesday,2018-12-11,HN,>37,F,current
1,28097592,4170276686,2020-01-01,20000.0,13,Chính chủ,600.0,2020-01,Wednesday,2019-12-30,HN,>37,F,current
2,47435144,4166729310,2020-01-01,10000.0,12,Chính chủ,200.0,2020-01,Wednesday,2019-11-11,HN,18_to_22,F,current
3,29080935,4174460303,2020-01-01,10000.0,13,Chính chủ,300.0,2020-01,Wednesday,2019-10-24,HN,18_to_22,F,current
4,14591075,4168216749,2020-01-01,10000.0,12,Chính chủ,200.0,2020-01,Wednesday,2019-12-28,Other Cities,18_to_22,F,current


In [42]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13576 entries, 0 to 13575
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   user_id          13576 non-null  object        
 1   order_id         13576 non-null  object        
 2   Date             13576 non-null  datetime64[ns]
 3   Amount           13576 non-null  float64       
 4   Merchant_id      13576 non-null  object        
 5   Purchase_status  13576 non-null  object        
 6   revenue          13576 non-null  float64       
 7   Month            13576 non-null  period[M]     
 8   DayOfWeek        13576 non-null  object        
 9   First_tran_date  13576 non-null  datetime64[ns]
 10  Location         12810 non-null  object        
 11  Age              11035 non-null  object        
 12  Gender           13576 non-null  object        
 13  type_user        13576 non-null  object        
dtypes: datetime64[ns](2), float64(2), obje

In [43]:
merged_df['type_user'].value_counts()

type_user
current    13102
new          474
Name: count, dtype: int64

In [44]:
# Calculate the total number of new users in December 2020.
Dec20_trans = merged_df[(merged_df['Date'].dt.year == 2020) & (transactions['Date'].dt.month == 12)]
Dec20_newUsers = len(Dec20_trans[Dec20_trans['type_user'] == "new"]["type_user"])
print(" The total number of new users in December 2020: ", Dec20_newUsers)

 The total number of new users in December 2020:  43
